In [1]:
!pip install -U pyabsa

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)


In [2]:
!pip show pyabsa

Name: pyabsa
Version: 2.4.1.post1
Summary: This tool provides the state-of-the-art models for aspect term extraction (ATE), aspect polarity classification (APC), and text classification (TC).
Home-page: https://github.com/yangheng95/PyABSA
Author: Yang, Heng
Author-email: hy345@exeter.ac.uk
License: MIT
Location: /opt/homebrew/anaconda3/envs/pytorch/lib/python3.12/site-packages
Requires: autocuda, boostaug, findfile, gitpython, metric-visualizer, networkx, pandas, protobuf, pytorch-warmup, sentencepiece, seqeval, spacy, termcolor, torch, tqdm, transformers, typing-extensions, update-checker
Required-by: boostaug


In [ ]:
# !pip uninstall transformers -y
# !pip install transformers==4.29.2

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
  Using cached transformers-4.29.2-py3-none-any.whl.metadata (112 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [62 lines of output]
      /private/var/folders/fh/_tq_jbr50895yx1hkbgpct5m0000gn/T/pip-build-env-id4j44nr/overlay/lib/python3.12/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in fav

In [3]:
import pandas as pd
import re
import emoji
from langdetect import detect
from pyabsa import AspectPolarityClassification as APC
import dask.dataframe as dd
import dask
from tqdm import tqdm

No CUDA GPU found in your device


/bin/sh: nvidia-smi: command not found


[2025-04-30 17:30:36] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






/opt/homebrew/anaconda3/envs/pytorch/lib/python3.12/multiprocessing/pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


In [4]:
csv_path = '../data/cleaned_reviews_with_sentiment.csv'
# Load the CSV file with Dask (lazy evaluation)
df = dd.read_csv(csv_path)

KeyboardInterrupt: 

In [ ]:
## 2. Preprocessing for the Aspect Polarity Classification
import re
import emoji

def clean_text(text):
    text = str(text)
    text = re.sub(r'<.*?>', '', text)                       # Remove HTML
    text = re.sub(r'http\S+|www\.\S+', '', text)            # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)                     # Remove emails
    text = emoji.demojize(text)                             # Convert emojis to :heart:, etc.
    return text.strip()
    
df['text'] = df['text'].apply(clean_text)

In [18]:
# Step 2: Sample a larger set for processing (e.g. 1000 rows)
df_sample = df.sample(n=100, random_state=42).copy()

# Step 3: Load the pre-trained sentiment classifier
classifier = APC.SentimentClassifier(checkpoint='english')

# Step 4: Define target aspects
aspects = ['look', 'smell', 'taste', 'feel']

# Step 5: Extract aspect-level sentiments
for aspect in aspects:
    print(f"\nProcessing aspect: {aspect}")
    sentiments = []
    for text in tqdm(df_sample['text']):
        try:
            result = classifier.predict(text=text, aspect=aspect)
            sentiments.append(result['sentiment'])
        except Exception as e:
            sentiments.append('unknown')  # fallback in case of error
    df_sample[f'{aspect}_sentiment'] = sentiments

# Step 6: Save results to new CSV
output_path = '../data/reviews_with_aspect_sentiment_test.csv'
df_sample.to_csv(output_path, index=False)
print(f"\n Saved to: {output_path}")

[2025-04-30 16:19:54] (2.4.1.post1) ********** Available APC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-30 16:19:54] (2.4.1.post1) ********** Available APC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-30 16:19:54] (2.4.1.post1) Downloading checkpoint:english 
[2025-04-30 16:19:54] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-04-30 16:19:54] (2.4.1.post1) Checkpoint already downloaded, skip
[2025-04-30 16:19:54] (2.4.1.post1) Load sentiment classifier from checkpoints/APC_ENGLISH_CHECKPOINT/fast_lsa_t_v2_English_acc_82.21_f1_81.81
[2025-04-30 16:19:54] (2.4.1.post1) config: checkpoints/APC_ENGLISH_CHECKPOINT/fast_lsa_t_v2_English_acc_82.21_f1_81.81/fast_lsa_t_v2.config
[2025-04-30 16:19:54] (2.4.1.post1) state_dict: checkpoints/APC_ENGLISH_CHECKPOINT/fast_lsa_t_v2_English_acc_82.21_f1_81.81/fast_lsa_t_v2.state_dict
[2025-04-30 16:19

  0%|          | 0/100 [00:00<?, ?it/s]

[2025-04-30 16:19:58] (2.4.1.post1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 1, text:  [B-ASP]Global Sentiment[E-ASP]    0%
[2025-04-30 16:19:58] (2.4.1.post1) Example 0: 0%


  1%|          | 1/100 [00:00<00:19,  5.14it/s]

[2025-04-30 16:19:58] (2.4.1.post1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 1, text:  [B-ASP]Global Sentiment[E-ASP]    Can: Poured a clear amber color le with a nice large off-white foamy head with good retention and some lacing. Aroma of dank hoppy notes with some fruity ester and light caramel notes is quite enticing. Taste is a nice mix between some dank hoppy notes with some fruity ester and some dry caramel notes. Body is about average with good carbonation. Solid amber ale with plenty of character.  
[2025-04-30 16:19:58] (2.4.1.post1) Example 0: Can: Poured a clear amber color le with a nice large off-white foamy head with good retention and some lacing. Aroma of dank hoppy notes with some fruity ester and light caramel notes is quite enticing. Taste is a nice mix between some dank hoppy notes with some fruity ester and some dry caramel notes. Body is about average with good carbonation. Solid amber ale with plenty of character.

...

[2025-04-30 16:21:10] (2.4.1.post1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 1, text:  [B-ASP]Global Sentiment[E-ASP]    this dubbel is mahogany in colour with a minimal head and no lace... it has super fast little carbonation bubbles...the nose holds spice and a rich malted aroma...smells somewhat vinous and dark fruit charactered with yeast inflections...the flavour is that of a dominant malt base with spicery and vinous impressions...yeasty tones as if bottle refermented...mouthfeel is not overly robust and tips in at the fat end of lightness approaching medium...drinkability is good as this is quite the mild little creation...finish is super clean as the label proclaims...I would go so far as to designate it as sessionable if one were to see fit as an admirer of the style... quite pleasant...nothing mind blowing at all, but solid...first Allagash for me...not the last...  
[2025-04-30 16:21:11] (2.4.1.post1) Example 0: this dubbel is

 99%|█████████▉| 99/100 [00:18<00:00,  5.55it/s]

[2025-04-30 16:21:11] (2.4.1.post1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 1, text:  [B-ASP]Global Sentiment[E-ASP]    473 ml can, from LCBO. Pours a very lightly hazed dark orange, some copper highlights, nice and thick white head forms, slowly settles down to a thick ring, some lace chunks stick to the glass. Smell is nice, very malty, signature Creemore breadiness, some toasty malts and caramel, earthy, some floral, nice light citrus hops, grassy, lightly solvent, but smells good. Taste is similar, dry and toasty up front, bready, caramel sweetness through the middle, very grassy hops, some citrus and floral, touch of fruit in the finish, almost a little tannic and grapey, nice dry finish, solid bitterness. Mouthfeel is light bodied with low to medium carbonation, just right. Nice lager with some great character. Looking forward to the hoppier batch 2.  
[2025-04-30 16:21:11] (2.4.1.post1) Example 0: 473 ml can, from LCBO. Pours a v

100%|██████████| 100/100 [00:18<00:00,  5.47it/s]



 Saved to: ../data/reviews_with_aspect_sentiment_test.csv


In [19]:
# Load the output file with aspect-level sentiment columns
df_result = pd.read_csv('../data/reviews_with_aspect_sentiment_test.csv')

,beer_id,username,date,text,look,smell,taste,feel,overall,score,sentiment,look_sentiment,smell_sentiment,taste_sentiment,feel_sentiment
0,101,Walnut,2016-03-09,0%,3.75,3.50,3.50,3.75,3.75,3.59,neutral,['Negative'],['Negative'],['Negative'],['Negative']
1,261565,Phyl21ca,2017-07-04,Can: Poured a clear amber color le with a n...,4.25,3.75,3.75,3.75,3.75,3.78,neutral,['Positive'],['Positive'],['Positive'],['Positive']
2,322435,agreenman19,2018-02-10,"L - ""oat cream IPA"". Jesus, OH, why you do ...",4.50,4.25,4.25,4.25,4.25,4.27,positive,['Neutral'],['Neutral'],['Neutral'],['Neutral']
3,55317,Mantooth,2012-01-31,Draft into a Samichlaus snifter. A: Orange ...,4.00,3.00,3.00,3.00,3.00,3.06,neutral,['Neutral'],['Neutral'],['Neutral'],['Neutral']
4,45552,Efay,2013-01-17,Deschutes took Chicago by storm this past m...,4.50,5.00,4.25,4.50,4.50,4.52,positive,['Positive'],['Positive'],['Positive'],['Positive']
5,23155,cvstrickland,2008-07-24,12-ounce bottle poured into a pint willi gl...,3.50,3.00,3.00,3.50,3.00,3.08,neutral,['Neutral'],['Neutral'],['Neutral'],['Neutral']
6,258879,Gavage,2017-02-12,Appearance: pours a crystal clear golden ye...,4.25,3.25,4.00,4.00,4.00,3.84,neutral,['Positive'],['Positive'],['Positive'],['Positive']
7,4074,hotstuff,2003-08-05,This beer had a medium sized head that cont...,2.50,3.00,3.00,2.50,3.00,2.92,neutral,['Neutral'],['Neutral'],['Neutral'],['Neutral']
8,1283,PlinyMeNow,2017-03-29,I went into it with low expectations and wa...,3.25,2.75,3.50,3.00,3.25,3.21,neutral,['Positive'],['Positive'],['Positive'],['Positive']
9,169622,worldsworstextra,2015-12-25,My favorite De Garde. No other brewery is a...,5.00,4.50,4.75,4.50,4.75,4.68,positive,['Positive'],['Positive'],['Positive'],['Positive']


In [20]:
# Show the first 10 rows
df_result.head(50)

,beer_id,username,date,text,look,smell,taste,feel,overall,score,sentiment,look_sentiment,smell_sentiment,taste_sentiment,feel_sentiment
0,101,Walnut,2016-03-09,0%,3.75,3.50,3.50,3.75,3.75,3.59,neutral,['Negative'],['Negative'],['Negative'],['Negative']
1,261565,Phyl21ca,2017-07-04,Can: Poured a clear amber color le with a n...,4.25,3.75,3.75,3.75,3.75,3.78,neutral,['Positive'],['Positive'],['Positive'],['Positive']
2,322435,agreenman19,2018-02-10,"L - ""oat cream IPA"". Jesus, OH, why you do ...",4.50,4.25,4.25,4.25,4.25,4.27,positive,['Neutral'],['Neutral'],['Neutral'],['Neutral']
3,55317,Mantooth,2012-01-31,Draft into a Samichlaus snifter. A: Orange ...,4.00,3.00,3.00,3.00,3.00,3.06,neutral,['Neutral'],['Neutral'],['Neutral'],['Neutral']
4,45552,Efay,2013-01-17,Deschutes took Chicago by storm this past m...,4.50,5.00,4.25,4.50,4.50,4.52,positive,['Positive'],['Positive'],['Positive'],['Positive']
5,23155,cvstrickland,2008-07-24,12-ounce bottle poured into a pint willi gl...,3.50,3.00,3.00,3.50,3.00,3.08,neutral,['Neutral'],['Neutral'],['Neutral'],['Neutral']
6,258879,Gavage,2017-02-12,Appearance: pours a crystal clear golden ye...,4.25,3.25,4.00,4.00,4.00,3.84,neutral,['Positive'],['Positive'],['Positive'],['Positive']
7,4074,hotstuff,2003-08-05,This beer had a medium sized head that cont...,2.50,3.00,3.00,2.50,3.00,2.92,neutral,['Neutral'],['Neutral'],['Neutral'],['Neutral']
8,1283,PlinyMeNow,2017-03-29,I went into it with low expectations and wa...,3.25,2.75,3.50,3.00,3.25,3.21,neutral,['Positive'],['Positive'],['Positive'],['Positive']
9,169622,worldsworstextra,2015-12-25,My favorite De Garde. No other brewery is a...,5.00,4.50,4.75,4.50,4.75,4.68,positive,['Positive'],['Positive'],['Positive'],['Positive']


In [29]:
for col in ['look_sentiment', 'smell_sentiment', 'taste_sentiment', 'feel_sentiment']:
    df[col] = df[col].apply(lambda x: eval(x)[0] if isinstance(x, str) else x)
    df[col] = df[col].str.lower()

df['sentiment'] = df['sentiment'].str.lower()

In [30]:
def aggregate_aspect_sentiment(row):
    aspects = [row['look_sentiment'], row['smell_sentiment'], row['taste_sentiment'], row['feel_sentiment']]
    pos = aspects.count('positive')
    neg = aspects.count('negative')
    if pos >= 2:
        return 'positive'
    elif neg >= 2:
        return 'negative'
    else:
        return 'neutral'

df['aggregated_aspect_sentiment'] = df.apply(aggregate_aspect_sentiment, axis=1)

In [31]:
df['match'] = df['sentiment'] == df['aggregated_aspect_sentiment']
accuracy = df['match'].mean()
print(f"Consistency between labeled sentiment and aspect-derived sentiment: {accuracy:.2%}")

Consistency between labeled sentiment and aspect-derived sentiment: 56.00%
